In [65]:
import torch
import torch.nn as nn
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm import tqdm 

In [66]:
# 예시 데이터로 대체 (이후 실제 데이터셋으로 변경해야 합니다)
dataset = pd.read_csv("../dataset/second_classification_data_neg_spell_check.csv", encoding='UTF-8')

In [67]:
# 데이터 프레임에서 한국어 문장과 라벨을 가져옵니다.
sentences = dataset['sentence'].values
labels = dataset['sentiment'].values

In [68]:
dataset['sentiment'].value_counts()

분노    33382
슬픔    29694
불안    24911
Name: sentiment, dtype: int64

In [69]:
# 라벨을 One-Hot 인코딩으로 변환합니다.
label_map = {'불안': 0, '슬픔': 1, '분노': 2}
labels = np.array([label_map[label] for label in labels])

In [70]:
# koELECTRA 토크나이저 불러오기
tokenizer = ElectraTokenizer.from_pretrained("koelectra-base-v3-discriminator")

In [71]:
# 문장을 토큰화하고 시퀀스로 변환했을 때의 길이 출력
length = [len(tokenizer.encode(sentence)) for sentence in tqdm(sentences)]
len(length)

100%|██████████| 87987/87987 [00:22<00:00, 3924.62it/s]


87987

In [72]:
import plotly.graph_objects as go

dataset['length'] = length

# 히스토그램 생성
histogram = go.Histogram(x=dataset['length'], nbinsx=20)

# 최댓값과 최솟값 계산
max_value = dataset['length'].max()
min_value = dataset['length'].min()

# 최댓값과 최솟값을 주석으로 추가하여 표시
annotations = [
    {
        'x': max_value,
        'y': 0,
        'xref': 'x',
        'yref': 'y',
        'text': f'Max: {max_value}',
        'showarrow': True,
        'arrowhead': 4,
        'ax': 0,
        'ay': -40
    },
    {
        'x': min_value,
        'y': 0,
        'xref': 'x',
        'yref': 'y',
        'text': f'Min: {min_value}',
        'showarrow': True,
        'arrowhead': 4,
        'ax': 0,
        'ay': -40
    }
]

# 그래프 생성 및 레이아웃 설정
fig = go.Figure(histogram)
fig.update_layout(annotations=annotations)

# 그래프 출력
fig.show()

In [73]:
len([i for i in length if i <= 40])/len(length) # 패딩 최대길이를 산정하기 위함. 40으로 설정하면 약 99%커버가능

0.9957721026969893

In [74]:
# 문장을 토큰화하고 시퀀스로 변환합니다.
sequences = [tokenizer.encode(sentence, padding='max_length', max_length=40, truncation=True) for sentence in tqdm(sentences)]

100%|██████████| 87987/87987 [00:23<00:00, 3718.06it/s]


In [75]:
# 학습 데이터와 검증 데이터로 나눕니다.
train_sequences, val_sequences, train_labels, val_labels = train_test_split(
    sequences, labels, test_size=0.2, random_state=42
)

In [76]:
# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_length):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long),
        }

In [77]:
# 데이터셋과 데이터로더 생성
train_dataset = CustomDataset(train_sequences, train_labels, tokenizer, max_length=40)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = CustomDataset(val_sequences, val_labels, tokenizer, max_length=40)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [78]:
# koELECTRA 모델 불러오기
model = ElectraForSequenceClassification.from_pretrained("koelectra-base-v3-discriminator", num_labels=3)

Some weights of the model checkpoint at koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at koelectra-base-v3-discriminator and are newly initialized: ['classifier.out_p

In [79]:
''' 모델가중치를 불러오거나 체크포인트를 불러올 때 실행 '''

# 모델의 상태 딕셔너리를 로드합니다.
model_state_dict = torch.load("neg_model_state_dict_epoch_10.pt")

# 모델을 생성하고 상태를 로드합니다.
model = ElectraForSequenceClassification.from_pretrained("koelectra-base-v3-discriminator", num_labels=3)
model.load_state_dict(model_state_dict)

# 옵티마이저의 상태 딕셔너리를 로드합니다.
optimizer_state_dict = torch.load("neg_optimizer_state_dict_epoch_10.pt")

# 옵티마이저를 생성하고 상태를 로드합니다.
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
optimizer.load_state_dict(optimizer_state_dict)

Some weights of the model checkpoint at koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at koelectra-base-v3-discriminator and are newly initialized: ['classifier.out_p

In [80]:
# 변경하고자 하는 Dropout 비율
new_dropout_rate = 0.2

# 기존의 Dropout 비율 변경
for name, module in model.electra.named_children():
    if "dropout" in name:
        module.p = new_dropout_rate

In [ ]:
# 모델 구조 확인
# print(model)

In [81]:
# 학습 함수 정의 (tqdm을 사용하여 진행 상황 및 지표 시각화)
def train_fn(data_loader, model, optimizer, device):
    model.train()
    progress_bar = tqdm(data_loader, desc="Training")
    
    train_losses = []  # train_loss 기록을 위한 리스트
    train_accs = []  # train_accuracy 기록을 위한 리스트

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()
        
        # 정확도 계산
        predicted_labels = torch.argmax(logits, dim=1)
        accuracy = (predicted_labels == labels).float().mean().item()
        
        train_losses.append(loss.item())
        train_accs.append(accuracy)
        progress_bar.set_postfix({'Loss': loss.item(), 'Accuracy': accuracy})

    return train_losses, train_accs

In [82]:
# 평가 함수 정의
def eval_fn(data_loader, model, device):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    predicted_labels_list = []  # 예측한 라벨들을 저장하기 위한 리스트
    true_labels_list = []  # 실제 라벨들을 저장하기 위한 리스트

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            predicted_labels = torch.argmax(logits, dim=1)
            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += labels.size(0)

            # 예측한 라벨과 실제 라벨을 리스트에 추가
            predicted_labels_list.extend(predicted_labels.tolist())
            true_labels_list.extend(labels.tolist())

    accuracy = correct_predictions / total_predictions

    # 예측한 라벨과 실제 라벨 출력
    predicted_labels_list = np.array(predicted_labels_list)
    true_labels_list = np.array(true_labels_list)
    print("Predicted Labels:", predicted_labels_list)
    print("True Labels:", true_labels_list)

    return accuracy

In [83]:
# 장치 설정 (GPU 사용을 위해)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# 옵티마이저와 손실 함수 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [84]:
# 학습 및 Early Stopping 정의
num_epochs = 10

def train_with_early_stopping(train_dataloader, val_dataloader, model, optimizer, loss_fn, device, patience=3):
    model.to(device)
    best_val_accuracy = 0.0
    best_model_state_dict = None
    no_improvement = 0

    for epoch in range(num_epochs):
        train_losses, train_accs = train_fn(train_dataloader, model, optimizer, loss_fn, device)
        val_loss, val_accuracy = eval_fn(val_dataloader, model, loss_fn, device)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {np.mean(train_losses):.4f} - Train Accuracy: {np.mean(train_accs):.4f} - Validation Loss: {val_loss:.4f} - Validation Accuracy: {val_accuracy:.4f}")

        # Early Stopping 체크
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state_dict = model.state_dict()
            no_improvement = 0
        else:
            no_improvement += 1

        if no_improvement >= patience:
            print(f"No improvement in validation accuracy for {patience} epochs. Early stopping...")
            break

    # 최적의 모델 state_dict 반환
    return best_model_state_dict

In [86]:
# 학습시작
train_losses_epoch = []  # epoch 별 train_loss 기록을 위한 리스트
train_accs_epoch = []  # epoch 별 train_accuracy 기록을 위한 리스트
for epoch in range(num_epochs):
    train_losses, train_accs = train_fn(train_dataloader, model, optimizer, device)
    train_losses_epoch.extend(train_losses)
    train_accs_epoch.extend(train_accs)
    # 일정한 간격으로 체크포인트 저장
    if (epoch-1) % 5 == 0 and epoch > 0:
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            # 필요한 경우 추가 정보 저장 (예: loss, accuracy 등)
        }
        torch.save(checkpoint, f'./checkpoint/checkpoint_epoch_20_{epoch}.pth')

Training: 100%|██████████| 1100/1100 [04:33<00:00,  4.02it/s, Loss=0.077, Accuracy=0.981] 


In [87]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=("Train Loss", "Train Accuracy"))

# Loss 그래프
fig.add_trace(
    go.Scatter(x=list(range(len(train_losses_epoch))), y=train_losses_epoch, mode='lines', name='Train Loss'),
    row=1, col=1
)
fig.update_xaxes(title_text="Iterations", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=1)

# Accuracy 그래프
fig.add_trace(
    go.Scatter(x=list(range(len(train_accs_epoch))), y=train_accs_epoch, mode='lines', name='Train Accuracy'),
    row=1, col=2
)
fig.update_xaxes(title_text="Iterations", row=1, col=2)
fig.update_yaxes(title_text="Accuracy", row=1, col=2)

fig.update_layout(title="Training Progress", showlegend=False)
fig.show()

In [88]:
# 모델의 상태 딕셔너리를 얻어옵니다.
model_state_dict = model.state_dict()

# 모델 상태 딕셔너리를 파일로 저장합니다.
torch.save(model_state_dict, "neg_model_state_dict_epoch_20.pt")

# 옵티마이저의 상태 딕셔너리를 얻어옵니다.
optimizer_state_dict = optimizer.state_dict()

# 옵티마이저 상태 딕셔너리를 파일로 저장합니다.
torch.save(optimizer_state_dict, "neg_optimizer_state_dict_epoch_20.pt")

In [89]:
# 검증 데이터로 평가 수행
val_accuracy = eval_fn(val_dataloader, model, device)
print("Validation Accuracy:", val_accuracy)

Predicted Labels: [0 2 1 ... 1 2 2]
True Labels: [0 0 1 ... 1 2 2]
Validation Accuracy: 0.7191726332537789


In [90]:
# 한국어 문장을 입력으로 받아서 예측 라벨을 출력하는 함수
def predict_label(sentence, model, tokenizer, device):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()
        return predicted_label

In [100]:
# 한국어 문장 입력 받기
korean_sentence = "o자 다리 많이 심한가요"

# 예측 라벨 출력
predicted_label = predict_label(korean_sentence, model, tokenizer, device)
emotion_labels = ['불안', '슬픔', '분노']
print("입력 문장:", korean_sentence)
print("예측 라벨:", emotion_labels[predicted_label])

입력 문장: o자 다리 많이 심한가요
예측 라벨: 불안
